In [ ]:
import sys

sys.path.insert(0,'/Users/jmordetsky/My Drive/ml-trde-notebooks')


Mounted at /content/drive


In [ ]:
!pip install -Uqq ipdb
import ipdb
%pdb off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.1/783.1 KB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 KB 36.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have ipython 8.9.0 which is incompatible.


Automatic pdb calling has been turned OFF


In [ ]:
# ***** WARNING : Install deps - This will BUILD TALib and takes a while!
%run -i '/content/drive/My Drive/ml-trde-notebooks/installs.ipynb'

In [ ]:

# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/common.ipynb'


Modified
1675568827
1675568827.0
1675568827.0


In [ ]:
load_all_models()

loading: lstm_cv
loading: lstm_coins_cv
loading: lstm_att_cv
loading: lstm_att_ohlcv
loading: lstm_cv_rvi
loading: lstm_cv_vwap
loading: lstm_ohlc
loading: svm_cv
loading: svm_cv_vwap
loading: lstm_xgb_cols
models loaded


# Back Tester


## data prep

In [ ]:
print(keras.__version__)

2.9.0


In [ ]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/backtest-common.ipynb'

Modified
1675735896
1675735896.0
1675735896.0


## back test function

In [ ]:
slippage = .01
fee = .0025
cash = 5000
delete_us = []
stop_at = -5

def predictions_to_df(df):
    columns_pred = []
    columns_mse = []
    for name in model_config["day_bar_models"]:
      columns_pred.append(name+"_pred")
      columns_mse.append(name+"_mse")
    return lists_to_new_df([columns_pred, columns_mse], ["Predicted", "MSE"], df)

def calc_profit(open, close, side):
  profit = (close - open) * side
  percent = (profit / open) * 100
  return percent

def make_trade_decision(product_predictions, current_trade):
  # Get the index of the maximum value (long)
  max_row = product_predictions.nlargest(1,['Est-Change'])


  # Get the index of the min value (short)
  min_row = product_predictions.nsmallest(1,['Est-Change'])

  #should we go long or short?
  trade_type = 0
  trade_product = ""
  trade_close = 0

  if (abs(max_row["Est-Change"].values[0]) > abs(min_row["Est-Change"].values[0])):
    trade_product = max_row["Product"].values[0]
    trade_type = 1 #long
    trade_close = max_row["Close"].values[0]

  if (abs(max_row["Est-Change"].values[0]) < abs(min_row["Est-Change"].values[0])):
    trade_product = min_row["Product"].values[0]
    trade_type = -1 #short
    trade_close = min_row["Close"].values[0]

  verbose("max pred buy: ", max_row["Product"].values[0], max_row["Est-Change"].values[0])
  verbose("max pred short: ", min_row["Product"].values[0], min_row["Est-Change"].values[0])
  verbose("selected: ", trade_product, trade_type)

  if (current_trade != None and current_trade[0] == trade_product and current_trade[1] == trade_type):
    # same trade, hold.
    verbose("holding: ", trade_product, trade_type, " at price:", trade_close)
    verbose("vs: ", current_trade[0], current_trade[1], " at price:", current_trade[2])
    performance = calc_profit(current_trade[2], trade_close, current_trade[1])
    verbose("right now this trade is: ", performance)
    if (performance <= stop_at):
      info("Stop executed: ", current_trade)
      return [trade_product, -99, trade_close]
    else:
      trade_type = 0
      info("Trade decision: ", trade_product, trade_type)
      return [trade_product, trade_type, current_trade[2]]
    
  else:
    info("Trade decision: ", trade_product, trade_type)
    return [trade_product, trade_type, trade_close]

def get_price_with_slippage(price):
  return price + (price * slippage) #always model positive slippage (should we change that for shorts?)

def make_ledger_row():
  ledger = pd.DataFrame()
  ledger["Date"] = []
  ledger["Product"] = []
  ledger["Side"] = []
  ledger["Action"]  = []
  ledger["Price"] = []
  ledger["Fee"] = []
  return ledger

def close_trade(ledger, current_trade, index, dataframes):
  assert(len(ledger)>0)
  df = dataframes[current_trade[0]]
  data_row = df.iloc[[index]]

  ledger_row = make_ledger_row()
  ledger_row["Date"] = data_row["Date"]
  ledger_row["Product"] = [current_trade[0]]
  ledger_row["Side"] = [current_trade[1]]
  ledger_row["Action"] = ["exit"]
  ledger_row["Price"] = [get_price_with_slippage(data_row["Close"].values[0])]
  ledger_row["Fee"] = [cash * fee]
  ledger = ledger.append(ledger_row)
  
  info("Closing trade: ", ledger.tail(1))
  return ledger


def open_trade(ledger, product, side, index, dataframes):

  
  df = dataframes[product]
  data_row = df.iloc[[index]]

  ledger_row = make_ledger_row()
  ledger_row["Date"] = data_row["Date"]
  ledger_row["Product"] = [product]
  ledger_row["Side"] = [side]
  ledger_row["Action"] = ["enter"]
  ledger_row["Price"] = [get_price_with_slippage(data_row["Close"].values[0])]
  ledger_row["Fee"] = [fee]
  ledger = ledger.append(ledger_row)
  assert(len(ledger)>0)
  info("Opening trade: ", ledger.tail(1))
  return ledger 

def get_consensus_prediction_for_product(row, product):
    debug("raw row for: ", product, ":", row)
    predictions_as_df = predictions_to_df(row)
    debug("predictions row: ", predictions_as_df)
    return consensus_prediction(predictions_as_df)

def calculate_percent_change(consensus, close):
    return (consensus - close) / (close * 100)
    
def create_trade_assessment_row(df, index, product):
  row = pd.DataFrame(df.iloc[index]).T
  prediction = get_consensus_prediction_for_product(row, product)
  close = row["Close"].values[0]
  percent = calculate_percent_change(prediction, close)
  return pd.DataFrame({"Product":[product], "Est-Change":[percent], "Close": [close]})

def backtest(product_data, backfill=30):
  start = 0
  index = start
  first_key = list(product_data.keys())[0]
  final = len(product_data[first_key])
  first_df = product_data[first_key]

  current_trade = None

  ledger = make_ledger_row() #creates an empty ledger
  assert(len(ledger) == 0)

  all_predictions = pd.DataFrame()

  
   
  #iterate over any dataframes supplied
  #iterate over any dataframes supplied
  while(index < final):
    # allow backfill bars before starting the test
    row = pd.DataFrame(first_df.iloc[index]).T
    info(index, " of ", final, " date:", row["Date"].values[0])
    if (index < backfill):
      index+=1
      continue

    product_predictions = pd.DataFrame()
    product_predictions["Product"] = []
    product_predictions["Est-Change"] = []
    product_predictions["Close"] = []

    for product, df in product_data.items():
      try:
        new_row = create_trade_assessment_row(df, index, product)
        product_predictions = product_predictions.append(new_row)   
      except Exception as inst:
        error("Failed on: ", product, " no predictions possibly?", " error:", inst)
        delete_us.append(product)

    # make a decision
    decision = make_trade_decision(product_predictions, current_trade)
    [product, direction, new_close] = decision
    # if there is a current open trade and decision is not hold, close it
    if (direction != 0 and direction != -99):
      if current_trade != None:
        ledger = close_trade(ledger, current_trade, index, product_data)      
      # execute the new trade, take into account slippage, cash, fees
      ledger = open_trade(ledger, product, direction, index, product_data)
      current_trade = decision
    elif direction == -99:
      ledger = close_trade(ledger, current_trade, index, product_data)      
      current_trade = None
    else:
      info("decision is hold")
    index+=1
  
  return ledger


In [ ]:
product_data = get_all_product_timeseries(5, 180)
product_data = append_predictions(product_data)


fetch for:  1EARTH-USDT
fetch for:  1INCH-USDT
fetch for:  2CRZ-USDT
fetch for:  AAVE-USDT
fetch for:  ABBC-USDT
fetch for:  ACA-USDT
making predictions
predicting: 1EARTH-USDT
predicting: 1INCH-USDT
predicting: 2CRZ-USDT
predicting: AAVE-USDT
predicting: ABBC-USDT
predicting: ACA-USDT


In [ ]:
ledger = backtest(product_data)
ledger

In [ ]:
df = ledger
df['trade_id'] = (df['Action'] == 'enter').cumsum()
df['entry_price'] = df.groupby('trade_id')['Price'].transform(lambda x: x.iloc[0])
df['exit_price'] = df.groupby('trade_id')['Price'].transform(lambda x: x.iloc[-1])
df['entry_date'] = df.groupby('trade_id')['Date'].transform(lambda x: x.iloc[0])
df['exit_date'] = df.groupby('trade_id')['Date'].transform(lambda x: x.iloc[-1])
df['profit'] = (df['exit_price'] - df['entry_price']) * df['Side']
df['percent'] = (df['profit'] / df['entry_price']) * 100
df = df[df['Action'] == 'enter']
df['balance'] = 5000
df['balance'] = df['balance'] * (1 + df['percent'] / 100) - (2 * df['Fee'])
for i in range(1, df.shape[0]):
    df['balance'].iloc[i] = df['balance'].iloc[i-1] * (1 + df['percent'].iloc[i] / 100) - (2 * df['Fee'].iloc[i])

now = datetime.now()
today = now.strftime("%Y-%m-%d")
df.to_csv(data_path+"/backtest-" + today + ".csv")
df

,Date,Product,Side,Action,Price,Fee,trade_id,entry_price,exit_price,entry_date,exit_date,profit,percent,balance
149,2022-09-12,AAVE-USDT,1.000000,enter,91.898890,0.002500,1,91.898890,83.800710,2022-09-12,2022-09-13,-8.098180,-8.812054,4559.392290
148,2022-09-13,ACA-USDT,1.000000,enter,0.239976,0.002500,2,0.239976,0.238663,2022-09-13,2022-09-14,-0.001313,-0.547138,4534.441120
147,2022-09-14,AAVE-USDT,1.000000,enter,84.499630,0.002500,3,84.499630,79.300150,2022-09-14,2022-09-15,-5.199480,-6.153258,4255.420272
146,2022-09-15,1EARTH-USDT,1.000000,enter,0.001960,0.002500,4,0.001960,0.001810,2022-09-15,2022-09-16,-0.000150,-7.676455,3928.749831
145,2022-09-16,ACA-USDT,1.000000,enter,0.240683,0.002500,5,0.240683,0.221594,2022-09-16,2022-09-18,-0.019089,-7.931179,3617.148642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,2023-01-29,AAVE-USDT,1.000000,enter,88.540640,0.002500,74,88.540640,81.946350,2023-01-29,2023-01-30,-6.594290,-7.447755,-1113.967671
9,2023-01-30,ABBC-USDT,1.000000,enter,0.075003,0.002500,75,0.075003,0.076245,2023-01-30,2023-01-31,0.001242,1.656343,-1132.423792
8,2023-01-31,1EARTH-USDT,-1.000000,enter,0.003870,0.002500,76,0.003870,0.003899,2023-01-31,2023-02-03,-0.000030,-0.767623,-1123.736051
5,2023-02-03,AAVE-USDT,1.000000,enter,92.065540,0.002500,77,92.065540,86.505490,2023-02-03,2023-02-05,-5.560050,-6.039230,-1055.876043


In [ ]:
# Lets try again, but instead of consensus use the highest mse